In [49]:
import symforce

symforce.set_symbolic_api("sympy")
symforce.set_log_level("warning")

from symforce.notebook_util import display
import symforce.symbolic as sf
import sys
sys.path.append('..')
import slam_utils as su


# Starts on page 76
# 1. Verify SO(3), SE(3), and Sim(3) are groups on matrix multiplication
a1 = sf.Rot3.symbolic('a1')
a2 = sf.Rot3.symbolic('a2')
a3 = sf.Rot3.symbolic('a3')
b1 = sf.Pose3_SE3.symbolic('b1')
b2 = sf.Pose3_SE3.symbolic('b2')
b3 = sf.Pose3_SE3.symbolic('b3')
s = sf.Symbol('s')
t1 = sf.V3.symbolic('t1')
t2 = sf.V3.symbolic('t2')
t3 = sf.V3.symbolic('t3')
R1 = sf.Rot3.symbolic('R1').to_rotation_matrix()
R2 = sf.Rot3.symbolic('R2').to_rotation_matrix()
R3 = sf.Rot3.symbolic('R3').to_rotation_matrix()
c1 = sf.M44.block_matrix([
    [s*R1, t1],
    [sf.V3.zero().transpose(), sf.V1(1)]
])
c2 = sf.M44.block_matrix([
    [s*R2, t2],
    [sf.V3.zero().transpose(), sf.V1(1)]
])
c3 = sf.M44.block_matrix([
    [s*R3, t3],
    [sf.V3.zero().transpose(), sf.V1(1)]
])

# Closure: 
display(type(a1*a2), type(b1*b2), type(c1*c2))

symforce.geo.rot3.Rot3

symforce.geo.pose3_se3.Pose3_SE3

symforce.geo.matrix.Matrix44

In [54]:
jjj# Combination
v1 = (a1*a2)*a3 
v2 = a1*(a2*a3)
display((v1.to_rotation_matrix()-v2.to_rotation_matrix()).simplify())
v1 = ((b1*b2)*b3).subs({b1.t[0]:0, b1.t[1]:0, b1.t[2]:0,
              b2.t[0]:0, b2.t[1]:0, b2.t[2]:0,
              b3.t[0]:0, b3.t[1]:0, b3.t[2]:0})
v2 = (b1*(b2*b3)).subs({b1.t[0]:0, b1.t[1]:0, b1.t[2]:0,
              b2.t[0]:0, b2.t[1]:0, b2.t[2]:0,
              b3.t[0]:0, b3.t[1]:0, b3.t[2]:0})
display((v1.to_homogenous_matrix() - v2.to_homogenous_matrix()).simplify())
v1 = (c1*c2)*c3
v2 = (c1*(c2*c3))
v3 = v1-v2
v3.simplify()

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

⎡0  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎣0  0  0  0⎦

⎡0  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎣0  0  0  0⎦

In [58]:
# unit
display(a1, sf.Rot3()*a1, a1*sf.Rot3())
display(b1, sf.Pose3_SE3()*b1, b1*sf.Pose3_SE3())
display(c1 - sf.I44()*c1 )

<Rot3 <Q xyzw=[a1_x, a1_y, a1_z, a1_w]>>

<Rot3 <Q xyzw=[a1_x, a1_y, a1_z, a1_w]>>

<Rot3 <Q xyzw=[a1_x, a1_y, a1_z, a1_w]>>

<Pose3_SE3 R=<Rot3 <Q xyzw=[b1.R_x, b1.R_y, b1.R_z, b1.R_w]>>, t=(b1.t0, b1.t1, b1.t2)>

<Pose3_SE3 R=<Rot3 <Q xyzw=[b1.R_x, b1.R_y, b1.R_z, b1.R_w]>>, t=(b1.t0, b1.t1, b1.t2)>

<Pose3_SE3 R=<Rot3 <Q xyzw=[b1.R_x, b1.R_y, b1.R_z, b1.R_w]>>, t=(b1.t0, b1.t1, b1.t2)>

⎡0  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎣0  0  0  0⎦

In [69]:
# Inverse
(a1*a1.inverse()).to_rotation_matrix()
(b1*b1.inverse()).to_homogenous_matrix().subs({b1.t[0]:0, b1.t[1]:0, b1.t[2]:0})
# (c1*c1.inv()) ugh, tried of messing with these

⎡1  0  0  0⎤
⎢          ⎥
⎢0  1  0  0⎥
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎣0  0  0  1⎦

In [80]:
#2. Verify R^3, R, x are a Lie algebra - did this in the text exercises
#3. Verify so(3) and se(3) are lie algebra = also did this in the text
#4. Verify properties 4.20 and 4.21, I'm assuming this means 3.20 and 3.21
#   which are some messy theta and n expressions. 
n = sf.V3.symbolic('n')
display(n.skew_symmetric(n)*n.skew_symmetric(n)-(n*n.transpose()-sf.I33()))
# note that n is unit vector so n0^2 + n1^2 + n2^2 = 1, so the displayed matrix is 
# actually all zeros.
e = n.skew_symmetric(n)*n.skew_symmetric(n)*n.skew_symmetric(n)+ n.skew_symmetric(n) 
e.simplify()

⎡    2     2     2                                                  ⎤
⎢- n₀  - n₁  - n₂  + 1            0                      0          ⎥
⎢                                                                   ⎥
⎢                           2     2     2                           ⎥
⎢          0            - n₀  - n₁  - n₂  + 1            0          ⎥
⎢                                                                   ⎥
⎢                                                  2     2     2    ⎥
⎣          0                      0            - n₀  - n₁  - n₂  + 1⎦

⎡                                ⎛  2     2     2    ⎞      ⎛    2     2     2
⎢            0                n₂⋅⎝n₀  + n₁  + n₂  - 1⎠   n₁⋅⎝- n₀  - n₁  - n₂ 
⎢                                                                             
⎢   ⎛    2     2     2    ⎞                                  ⎛  2     2     2 
⎢n₂⋅⎝- n₀  - n₁  - n₂  + 1⎠              0                n₀⋅⎝n₀  + n₁  + n₂  
⎢                                                                             
⎢    ⎛  2     2     2    ⎞      ⎛    2     2     2    ⎞                       
⎣ n₁⋅⎝n₀  + n₁  + n₂  - 1⎠   n₀⋅⎝- n₀  - n₁  - n₂  + 1⎠              0        

    ⎞⎤
 + 1⎠⎥
     ⎥
   ⎞ ⎥
- 1⎠ ⎥
     ⎥
     ⎥
     ⎦

In [136]:
# 5: Show that Rp^R^(T) = (Rp)^
R = sf.Rot3.symbolic('R').to_rotation_matrix()
p = sf.V3.symbolic('p')
R*p.skew_symmetric(p)*R.transpose() - (R*p).skew_symmetric(R*p)
# I give up on this one lol, can't figure it out. 

⎡                                                                 ⎛           
⎢                                          (2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z)⋅⎝-p₀⋅(-2⋅R_w
⎢                                                                             
⎢                                                             ⎛      2        
⎢-p₀⋅(-2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z) - p₁⋅(2⋅R_w⋅Rₓ + 2⋅R_y⋅R_z) - p₂⋅⎝- 2⋅Rₓ  - 2⋅R_y
⎢                                                                             
⎢                               ⎛      2        2    ⎞                        
⎣p₀⋅(2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) + p₁⋅⎝- 2⋅Rₓ  - 2⋅R_z  + 1⎠ + p₂⋅(-2⋅R_w⋅Rₓ + 2⋅R_y

                      ⎛       2        2    ⎞⎞                           ⎛    
⋅R_z + 2⋅Rₓ⋅R_y) + p₁⋅⎝- 2⋅R_y  - 2⋅R_z  + 1⎠⎠ + (-2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y)⋅⎝p₀⋅(
                                                                              
2    ⎞                          ⎛     ⎛      2        2    ⎞                  
  + 1⎠ + (2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z)⋅⎝- p₀⋅⎝- 2⋅Rₓ  - 2⋅

In [149]:
# 6: Show R*exp(p^)*R^T = exp((R*p)^)
v1 = R*p.skew_symmetric(p).from_tangent(p.skew_symmetric(p))*R.transpose()
v2 = ((R*p).skew_symmetric(R*p)).from_tangent((R*p).skew_symmetric(R*p))
(v1 - v2).simplify()

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢      ⎛     2            2             2            3        2            2  
⎢4⋅R_z⋅⎝- R_w ⋅Rₓ⋅p₀ - R_w ⋅R_y⋅p₁ - R_w ⋅R_z⋅p₂ - Rₓ ⋅p₀ - Rₓ ⋅R_y⋅p₁ - Rₓ ⋅R
⎢                                                                             
⎢       ⎛   2            2             2            3        2            2   
⎣ 4⋅R_y⋅⎝R_w ⋅Rₓ⋅p₀ + R_w ⋅R_y⋅p₁ + R_w ⋅R_z⋅p₂ + Rₓ ⋅p₀ + Rₓ ⋅R_y⋅p₁ + Rₓ ⋅R_

                                                                              
              0                                                               
                                                                              
              2            2                 3         2                 2    
_z⋅p₂ - Rₓ⋅R_y ⋅p₀ - Rₓ⋅R_z ⋅p₀ + Rₓ⋅p₀ - R_y ⋅p₁ -